In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ktrain
!pip install tf_keras
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
import ktrain
from ktrain import text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 87.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 63.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 41.8 MB/s eta 0:00:00
  Created wheel for ktrain: filename=ktrain-0.41.4-py3-none-any.whl size=25316932 sha256=63c12bd00f830792a3a870466a4e6224582e7a5427c1a520ae591fc31c81eac7
  Stored in directory: /root/.cache/pip/wheels/fa/6a/9c/8a873b38bbd8bc207d33c64726bd18f7ef85f8e70dc3ac2e4b
  Created wheel for keras_bert: filename=ke

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
tamil_stopwords = set([
    "அங்கு", "அங்கே", "அடுத்த", "அதற்கு", "அதனால்", "அதன்", "அதிக", "அதில்", "அது",
    "அதே", "அதை", "அந்த", "அந்தக்", "அந்தப்", "அல்லது", "அவரது", "அவர்", "அவர்கள்",
    "அவள்", "அவன்", "அவை", "அன்று", "ஆகிய", "ஆகியோர்", "ஆகும்", "ஆனால்", "இங்கு",
    "இங்கே", "இடத்தில்", "இடம்", "இதற்கு", "இதனால்", "இதனை", "இதன்", "இதில்", "இது",
    "இதை", "இந்த", "இந்தக்", "இந்தத்", "இந்தப்", "இப்போது", "இரு", "இருக்கும்", "இருந்த",
    "இருந்தது", "இருந்து", "இல்லை", "இவர்", "இவை", "இன்னும்", "உள்ள", "உள்ளது", "உள்ளன",
    "உன்", "எந்த", "எல்லாம்", "என", "எனக்", "எனக்கு", "எனப்படும்", "எனவும்", "எனவே",
    "எனினும்", "எனும்", "என்", "என்பது", "என்பதை", "என்ற", "என்று", "என்றும்", "என்ன",
    "என்னும்", "ஏன்", "ஒரு", "ஒரே", "ஓர்", "கொண்ட", "கொண்டு", "கொள்ள", "சற்று", "சில",
    "சிறு", "சேர்ந்த", "தவிர", "தனது", "தன்", "தான்", "நாம்", "நான்", "நீ", "பல",
    "பலரும்", "பல்வேறு", "பற்றி", "பற்றிய", "பிற", "பிறகு", "பின்", "பின்னர்", "பெரும்",
    "பேர்", "போது", "போல", "போல்", "போன்ற", "மட்டுமே", "மட்டும்", "மற்ற", "மற்றும்",
    "மிக", "மிகவும்", "மீது", "முதல்", "முறை", "மேலும்", "மேல்", "யார்", "வந்த",
    "வந்து", "வரும்", "வரை", "வரையில்", "விட", "விட்டு", "வேண்டும்", "வேறு"
])

In [ ]:
import re
import string

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Zஅஆஇஈஉஊஎஏஐஒஓௌகெஙடபணரஎழைசாௗ\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in tamil_stopwords]
    return " ".join(tokens)

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Fake News Detection in Dravidian Languages/Task 1 (Fake or Original)/Fake_train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/Fake News Detection in Dravidian Languages/Task 1 (Fake or Original)/Fake_dev.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Fake News Detection in Dravidian Languages/Task 1 (Fake or Original)/Fake_test_without_labels.csv')

In [ ]:
train_df.head(1)

,text,label
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake


In [ ]:
train_text = train_df['text']
train_label = train_df['label']
val_text = val_df['text']
val_label = val_df['label']
test_text = test_df['text']

In [ ]:
# train_text = [preprocess_text(text) for text in train_text]
# val_text = [preprocess_text(text) for text in val_text]
# test_text = [preprocess_text(text) for text in test_text]

In [ ]:
train_df['label'].value_counts()

,count
label,
original,1657
Fake,1599
original,1


In [ ]:
classes_list = ["original", "Fake", "original "]
train_label = train_label.apply(classes_list.index)
val_label = val_label.apply(classes_list.index)

In [ ]:
MODEL_NAME = 'FacebookAI/xlm-roberta-base'
t = text.Transformer(MODEL_NAME, maxlen=30, class_names=classes_list)
trn = t.preprocess_train(np.array(train_text), np.array(train_label))
val = t.preprocess_test(np.array(val_text), np.array(val_label))
model = t.get_classifier()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

preprocessing train...
language: ml
train sequence lengths:
	mean : 11
	95percentile : 35
	99percentile : 70


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: ml
test sequence lengths:
	mean : 11
	95percentile : 30
	99percentile : 66


In [ ]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = "Tam_Muril"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
learner.fit_onecycle(2e-5, 10, verbose=2, callbacks=callbacks_list)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10

Epoch 1: val_accuracy improved from -inf to 0.66503, saving model to Tam_Muril
102/102 - 108s - loss: 0.8823 - accuracy: 0.4943 - val_loss: 0.6908 - val_accuracy: 0.6650 - 108s/epoch - 1s/step
Epoch 2/10

Epoch 2: val_accuracy improved from 0.66503 to 0.78528, saving model to Tam_Muril
102/102 - 75s - loss: 0.6671 - accuracy: 0.6371 - val_loss: 0.4868 - val_accuracy: 0.7853 - 75s/epoch - 736ms/step
Epoch 3/10

Epoch 3: val_accuracy did not improve from 0.78528
102/102 - 32s - loss: 0.5033 - accuracy: 0.7716 - val_loss: 0.4396 - val_accuracy: 0.7828 - 32s/epoch - 311ms/step
Epoch 4/10

Epoch 4: val_accuracy improved from 0.78528 to 0.83313, saving model to Tam_Muril
102/102 - 99s - loss: 0.4086 - accuracy: 0.8189 - val_loss: 0.3833 - val_accuracy: 0.8331 - 99s/epoch - 968ms/step
Epoch 5/10

Epoch 5: val_accuracy did not improve from 0.83313
102/102 - 32s - loss: 0.4221 - accuracy: 0.8078 - val_loss: 0.4126 - val_

In [ ]:
learner.validate(class_names=classes_list[:2]) # Use only the first two elements of classes_list

26/26 [==============================] - 2s 86ms/step
              precision    recall  f1-score   support

    original       0.87      0.84      0.85       409
        Fake       0.84      0.87      0.86       406

    accuracy                           0.86       815
   macro avg       0.86      0.86      0.86       815
weighted avg       0.86      0.86      0.86       815



array([[343,  66],
       [ 51, 355]])

In [ ]:
test_text1 = test_df['text']
model.load_weights(filepath)
predictor = ktrain.get_predictor(learner.model, preproc=t)
predict = predictor.predict(test_text1.values)

In [ ]:
test_df.head(1)

,Id,text
0,Fake_01,5000 ഉള്ള പോൾ ലോഗ്‌ഡ്‌വൻ ഇപ്പോള് 250000 എന്താ...


In [ ]:
df = pd.DataFrame({
    'Id': test_df['Id'],
    'Labels': predict
})

In [ ]:
import csv
df.to_csv("/content/drive/MyDrive/Fake News Detection in Dravidian Languages/Task 1 (Fake or Original)/MNLP_task1_run1.csv", index=False)
check_submission = pd.read_csv("/content/drive/MyDrive/Fake News Detection in Dravidian Languages/Task 1 (Fake or Original)/MNLP_task1_run1.csv")
check_submission.head()

,Id,Labels
0,Fake_01,Fake
1,Fake_02,Fake
2,Fake_03,original
3,Fake_04,original
4,Fake_05,original


In [ ]:
df['Labels'].value_counts()

,count
Labels,
original,552
Fake,467


In [ ]:
predict

['Fake',
 'Fake',
 'original',
 'original',
 'original',
 'Fake',
 'original',
 'original',
 'original',
 'original',
 'original',
 'original',
 'Fake',
 'original',
 'original',
 'Fake',
 'original',
 'original',
 'Fake',
 'Fake',
 'original',
 'Fake',
 'Fake',
 'Fake',
 'original',
 'Fake',
 'Fake',
 'Fake',
 'Fake',
 'original',
 'Fake',
 'original',
 'original',
 'Fake',
 'original',
 'original',
 'original',
 'Fake',
 'Fake',
 'Fake',
 'original',
 'Fake',
 'Fake',
 'Fake',
 'original',
 'original',
 'Fake',
 'Fake',
 'Fake',
 'Fake',
 'original',
 'original',
 'Fake',
 'Fake',
 'Fake',
 'original',
 'Fake',
 'Fake',
 'original',
 'Fake',
 'Fake',
 'original',
 'original',
 'Fake',
 'original',
 'Fake',
 'original',
 'original',
 'original',
 'original',
 'original',
 'original',
 'original',
 'original',
 'original',
 'Fake',
 'Fake',
 'Fake',
 'Fake',
 'Fake',
 'original',
 'Fake',
 'original',
 'original',
 'original',
 'Fake',
 'original',
 'original',
 'original',
 'original'